In [1]:
"""
The VertiSync Scheduling Policy from the following paper:
https://ieeexplore.ieee.org/abstract/document/10422121

Author: Milad Pooladsanj, 2024
"""

# Import PuLP modeler functions
from pulp import *
import collections
import time
path_to_gurobi = r'C:\gurobi1101\win64\bin\gurobi_cl.exe'

In [6]:
vertiports = ["HB", "LB", "DTLA1", "DTLA2"]
OD_pairs = ["HB_DTLA1", "HB_DTLA2", "LB_DTLA1", "LB_DTLA2",
            "DTLA2_HB", "DTLA1_LB", "HB_LB", "LB_HB"
           ]
OD_Dest = {"HB": ["DTLA2_HB", "LB_HB"],
           "LB": ["DTLA1_LB", "HB_LB"],
           "DTLA1": ["HB_DTLA1", "LB_DTLA1"],
           "DTLA2": ["HB_DTLA2", "LB_DTLA2"]
          }
OD_Origin = {"HB": ["HB_DTLA1", "HB_DTLA2", "HB_LB"],
             "LB": ["LB_DTLA1", "LB_DTLA2", "LB_HB"],
             "DTLA1": ["DTLA1_LB"],
             "DTLA2": ["DTLA2_HB"]
            }
demand = {"HB_DTLA1": 1, "HB_DTLA2": 1, "LB_DTLA1": 1, "LB_DTLA2": 1,
         "DTLA2_HB": 1, "DTLA1_LB": 0, "HB_LB": 0, "LB_HB": 0
         }
# Separation margin during takeoff and landing
tau = 5 #minutes

# Separation margin in air
tau_c = 0.5 #minutes

# k_{\tau}
k_tau = int(tau / tau_c)

# Number of aircraft
num_aircraft = 4
aircraft_list = ["a" + str(i) for i in range(num_aircraft)]

# An upper-bound on the length of the cycle
upper_cycle_time = 100 # \tau_c
time_list = ["n" + str(i) for i in range(upper_cycle_time)]

# Time it takes to traverse each O-D pair
time_to_traverse = {"HB_DTLA1": 8, "HB_DTLA2": 8, "LB_DTLA1": 8, "LB_DTLA2": 8,
                    "DTLA2_HB": 8, "DTLA1_LB": 8, "HB_LB": 6, "LB_HB": 6
                   }

# Number of slots associated with each O-D pair
num_slots = collections.Counter()
slot_list = collections.defaultdict(list)
for od in OD_pairs:
    num_slots[od] = int(time_to_traverse[od] / tau_c)
    slot_list[od] = ["s" + str(i) for i in range(num_slots[od])]


In [7]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("VertiSync Policy", LpMinimize)

# Create a list of tuples containing all slots for each aircraft/route
#time_aircraft_tuple = [(n, a) for n in range(upper_cycle_time) for a in range(num_aircraft)]
#for od in OD_pairs:
#    slots.extend((od, i, n, a) 
#                 for i in range(num_slots[od])
#                 for n in range()
#                 for a in range(num_aircraft)
#                ) 
    
# A dictionary called 'w' is created to contain the decision variables w_{i,n}^{a,p}
w = {}
for p in OD_pairs:
    temp = LpVariable.dicts("w", ([p], slot_list[p], time_list, aircraft_list), 0, None, LpInteger)
    w.update(temp) 

# A dictionary called 'wd' is created to contain the decision variables w_{d,n}^{a,p}
wd = {}
for p in OD_pairs:
    temp = LpVariable.dicts("wd", ([p], time_list, aircraft_list), 0, None, LpInteger)
    wd.update(temp) 

# A dictionary called 'wo' is created to contain the decision variables w_{o,n}^{a,p}
wo = {}
for p in OD_pairs:
    temp = LpVariable.dicts("wo", ([p], time_list, aircraft_list), 0, None, LpInteger)
    wo.update(temp) 
    
# A dictionary called 'u_van' is created to contain the decision variables u_{v, n}^{a}
u = LpVariable.dicts("u", (vertiports, time_list, aircraft_list),  cat="Binary")
# A dictionary called 'z_van' is created to contain the decision variables z_{v, n}^{a}
z = LpVariable.dicts("z", (vertiports, time_list, aircraft_list),  cat="Binary")

In [8]:
prob = LpProblem("VertiSync Policy", LpMinimize)
# Objective function    

prob += (
    lpSum([(wo[p][time_list[n]][a] - wo[p][time_list[n-1]][a]) * time_to_traverse[p]
          for p in OD_pairs for n in range(1,upper_cycle_time) for a in aircraft_list]),
         "Sum_of_Travel_Times",
) 

# Constraints
# Service all trip requests (2a)
for p in OD_pairs: 
    prob += lpSum(wo[p][time_list[-1]][a] for a in aircraft_list) >= demand[p]

# w, wo, wd must be non-decreasing over time (2b)
for p in OD_pairs:
    for n in range(1, upper_cycle_time):
        for a in aircraft_list:
            prob += wo[p][time_list[n]][a] - wo[p][time_list[n-1]][a] >= 0
            prob += wd[p][time_list[n]][a] - wd[p][time_list[n-1]][a] >= 0
            for i in slot_list[p]:
                prob += w[p][i][time_list[n]][a] - w[p][i][time_list[n-1]][a] >= 0

# Each aircraft must occupy at most one slot at any time (2c)
"""
for n in range(1, upper_cycle_time):
    for a in aircraft_list:
        prob += (
            lpSum(w[p][i][time_list[n]][a] - w[p][i][time_list[n-1]][a] for p in OD_pairs for i in slot_list[p]) + 
            lpSum(wo[p][time_list[n]][a] - wo[p][time_list[n-1]][a] for p in OD_pairs) + 
            lpSum(wd[p][time_list[n]][a] - wd[p][time_list[n-1]][a] for p in OD_pairs) <= 1
        )
"""
# Each flying aircraft moves to the next slot at the next time step (2d)
for p in OD_pairs:
    for n in range(1, upper_cycle_time):
        for a in aircraft_list:
            prob += w[p][slot_list[p][-1]][time_list[n-1]][a] == wd[p][time_list[n]][a]
            prob += wo[p][time_list[n-1]][a] == w[p][slot_list[p][0]][time_list[n]][a]
            for i in range(num_slots[p] - 1):
                prob += w[p][slot_list[p][i]][time_list[n-1]][a] == w[p][slot_list[p][i+1]][time_list[n]][a]

# At most one aircraft must occupy overlapping slots (2e)
p = "HB_DTLA1"
q = "HB_DTLA2"
for i in range(num_slots[p]):
    for n in range(1, upper_cycle_time):
        prob += (
            lpSum(w[p][slot_list[p][i]][time_list[n]][a] - w[p][slot_list[p][i]][time_list[n-1]][a] +
                  w[q][slot_list[q][i]][time_list[n]][a] - w[q][slot_list[q][i]][time_list[n-1]][a]
                  for a in aircraft_list) <= 1
        )
r = "LB_DTLA1"
s = "LB_DTLA2"
for i in range(num_slots[r]):
    for n in range(1, upper_cycle_time):
        prob += (
            lpSum(w[r][slot_list[r][i]][time_list[n]][a] - w[r][slot_list[r][i]][time_list[n-1]][a] +
                  w[s][slot_list[s][i]][time_list[n]][a] - w[s][slot_list[s][i]][time_list[n-1]][a]
                  for a in aircraft_list) <= 1
        )
for i in range(6, 12):
    for n in range(1, upper_cycle_time):
        prob += (
            lpSum(w[p][slot_list[p][i]][time_list[n]][a] - w[p][slot_list[p][i]][time_list[n-1]][a] +
                  w[q][slot_list[q][i]][time_list[n]][a] - w[q][slot_list[q][i]][time_list[n-1]][a] +
                  w[r][slot_list[r][i]][time_list[n]][a] - w[r][slot_list[r][i]][time_list[n-1]][a] +
                  w[s][slot_list[s][i]][time_list[n]][a] - w[s][slot_list[s][i]][time_list[n-1]][a]
                  for a in aircraft_list) <= 1
        )

# Takeoff separation must be satisfiesd at every vertiport (2f)
for n in range(k_tau, upper_cycle_time):
    prob += (
        lpSum(wo[p][time_list[n]][a] - wo[p][time_list[n-k_tau]][a] +
              wo[q][time_list[n]][a] - wo[q][time_list[n-k_tau]][a] +
              wo["HB_LB"][time_list[n]][a] - wo["HB_LB"][time_list[n-k_tau]][a]
              for a in aircraft_list) <= 1
    )
    prob += (
        lpSum(wo[r][time_list[n]][a] - wo[r][time_list[n-k_tau]][a] +
              wo[s][time_list[n]][a] - wo[s][time_list[n-k_tau]][a] +
              wo["LB_HB"][time_list[n]][a] - wo["LB_HB"][time_list[n-k_tau]][a]
              for a in aircraft_list) <= 1
    )
    prob += (
        lpSum(wo["DTLA2_HB"][time_list[n]][a] - wo["DTLA2_HB"][time_list[n-k_tau]][a]
              for a in aircraft_list) <= 1
    )
    prob += (
        lpSum(wo["DTLA1_LB"][time_list[n]][a] - wo["DTLA1_LB"][time_list[n-k_tau]][a]
              for a in aircraft_list) <= 1
    )

# Takeoffs must occur only after an aircraft is available at a vertiport (2g)
# After takeoff, aircraft availability at a veriport is set to zero (2h)
# After landing, aircraft availability at a veriport is set to one (2j)
for v in vertiports:
    for n in range(k_tau, upper_cycle_time):
        for a in aircraft_list:
            prob += (
                lpSum(wo[p][time_list[n]][a] - wo[p][time_list[n-1]][a]
                     for p in OD_Origin[v]) <= 
                     u[v][time_list[n-k_tau]][a]
            )
            
            prob += (
                u[v][time_list[n-k_tau+1]][a] ==
                u[v][time_list[n-k_tau]][a] -
                lpSum(wo[p][time_list[n]][a] - wo[p][time_list[n-1]][a]
                     for p in OD_Origin[v]) +
                lpSum(wd[p][time_list[n-k_tau+1]][a] - wd[p][time_list[n-k_tau]][a]
                     for p in OD_Dest[v]
                )
            )

# Landing separation must be satisfiesd at every vertiport (2i)
p = "HB_DTLA1"
q = "HB_DTLA2"
r = "LB_DTLA1"
s = "LB_DTLA2"
for n in range(k_tau, upper_cycle_time):
    prob += (
        lpSum(wo[p][time_list[n]][a] - wo[p][time_list[n-1]][a] +
              wo[q][time_list[n]][a] - wo[q][time_list[n-1]][a] +
              wo["HB_LB"][time_list[n]][a] - wo["HB_LB"][time_list[n-1]][a] +
              wd["DTLA2_HB"][time_list[n-1]][a] - wd["DTLA2_HB"][time_list[n-k_tau]][a] +
              wd["LB_HB"][time_list[n-1]][a] - wd["LB_HB"][time_list[n-k_tau]][a]
              for a in aircraft_list) <= 1
    )
    prob += (
        lpSum(wo[r][time_list[n]][a] - wo[r][time_list[n-1]][a] +
              wo[s][time_list[n]][a] - wo[s][time_list[n-1]][a] +
              wo["LB_HB"][time_list[n]][a] - wo["LB_HB"][time_list[n-1]][a] +
              wd["DTLA1_LB"][time_list[n-1]][a] - wd["DTLA1_LB"][time_list[n-k_tau]][a] +
              wd["HB_LB"][time_list[n-1]][a] - wd["HB_LB"][time_list[n-k_tau]][a]
              for a in aircraft_list) <= 1
    )
    prob += (
        lpSum(wo["DTLA2_HB"][time_list[n]][a] - wo["DTLA2_HB"][time_list[n-1]][a] +
              wd[q][time_list[n-1]][a] - wd[q][time_list[n-k_tau]][a] +
              wd[s][time_list[n-1]][a] - wd[s][time_list[n-k_tau]][a]
              for a in aircraft_list) <= 1
    )
    prob += (
        lpSum(wo["DTLA1_LB"][time_list[n]][a] - wo["DTLA1_LB"][time_list[n-1]][a] +
              wd[p][time_list[n-1]][a] - wd[p][time_list[n-k_tau]][a] +
              wd[r][time_list[n-1]][a] - wd[r][time_list[n-k_tau]][a]
              for a in aircraft_list) <= 1
    )

# Initial conditions
# w, wo, wd are set to zero initially
for p in OD_pairs:
    for a in aircraft_list:
        for n in range(k_tau):
            prob += wo[p][time_list[n]][a] == 0
            prob += wd[p][time_list[n]][a] == 0
            for i in slot_list[p]:
                prob += w[p][i][time_list[n]][a] == 0

for i in range(num_aircraft):
    if i % 2 == 0:
        prob += u["HB"][time_list[0]][aircraft_list[i]] == 1
        prob += u["LB"][time_list[0]][aircraft_list[i]] == 0
    else:
        prob += u["HB"][time_list[0]][aircraft_list[i]] == 0
        prob += u["LB"][time_list[0]][aircraft_list[i]] == 1

for v in vertiports[2:]:
    for a in aircraft_list:
        prob += u[v][time_list[0]][a] == 0

In [9]:
start_time = time.time()

# Gurobi is chosen as the solver
solver = GUROBI_CMD(path=path_to_gurobi)

# The problem data is written to an .lp file
prob.writeLP("VertiSync.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve(solver)

# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])
print(prob.objective.value())

end_time = time.time()
duration = end_time - start_time
print(f"Time taken: {duration} seconds")

for v in prob.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)

Status: Optimal
40.0
Time taken: 4.468522310256958 seconds
u_DTLA1_n36_a0 = 1.0
u_DTLA1_n37_a0 = 1.0
u_DTLA1_n38_a0 = 1.0
u_DTLA1_n39_a0 = 1.0
u_DTLA1_n40_a0 = 1.0
u_DTLA1_n41_a0 = 1.0
u_DTLA1_n42_a0 = 1.0
u_DTLA1_n43_a0 = 1.0
u_DTLA1_n44_a0 = 1.0
u_DTLA1_n45_a0 = 1.0
u_DTLA1_n46_a0 = 1.0
u_DTLA1_n47_a0 = 1.0
u_DTLA1_n48_a0 = 1.0
u_DTLA1_n49_a0 = 1.0
u_DTLA1_n50_a0 = 1.0
u_DTLA1_n51_a0 = 1.0
u_DTLA1_n52_a0 = 1.0
u_DTLA1_n53_a0 = 1.0
u_DTLA1_n54_a0 = 1.0
u_DTLA1_n55_a0 = 1.0
u_DTLA1_n56_a0 = 1.0
u_DTLA1_n57_a0 = 1.0
u_DTLA1_n58_a0 = 1.0
u_DTLA1_n59_a0 = 1.0
u_DTLA1_n60_a0 = 1.0
u_DTLA1_n61_a0 = 1.0
u_DTLA1_n62_a0 = 1.0
u_DTLA1_n63_a0 = 1.0
u_DTLA1_n64_a0 = 1.0
u_DTLA1_n65_a0 = 1.0
u_DTLA1_n66_a0 = 1.0
u_DTLA1_n67_a0 = 1.0
u_DTLA1_n68_a0 = 1.0
u_DTLA1_n69_a0 = 1.0
u_DTLA1_n70_a0 = 1.0
u_DTLA1_n71_a0 = 1.0
u_DTLA1_n72_a0 = 1.0
u_DTLA1_n73_a0 = 1.0
u_DTLA1_n74_a0 = 1.0
u_DTLA1_n75_a0 = 1.0
u_DTLA1_n76_a0 = 1.0
u_DTLA1_n77_a0 = 1.0
u_DTLA1_n78_a0 = 1.0
u_DTLA1_n79_a0 = 1.0
u_DTLA1_n80_a0 = 